In [0]:
text_file = sc.textFile("/FileStore/tables/spark.txt")
data = text_file.flatMap(lambda l : l.split())\
                .map(lambda word : (word, 1))\
                .reduceByKey(lambda x, y : x+y)\
                .filter(lambda pair : pair[1] >= 2)
data.collect()

Out[2]: [('Spark', 4), ('is', 3), ('a', 2), ('This', 2)]

In [0]:
data = data.map(lambda p : (p[1], p[0]))\
            .sortByKey(ascending=False)
data.collect()

Out[3]: [(4, 'Spark'), (3, 'is'), (2, 'a'), (2, 'This')]

In [0]:
mapLink = sc.parallelize([["PageA", "PageB"], ["PageA", "PageC"], ["PageB", "PageC"], ["PageC", "PageA"], ["PageD","PageC"]])
links = mapLink.groupByKey()
print(list((k, list(v)) for (k,v) in links.collect()))
ranks = links.map(lambda pairs : (pairs[0], 1))
print(ranks.collect())

[('PageA', ['PageB', 'PageC']), ('PageB', ['PageC']), ('PageC', ['PageA']), ('PageD', ['PageC'])]
[('PageA', 1), ('PageB', 1), ('PageC', 1), ('PageD', 1)]

In [0]:
def computeContribs(urls, rank) :
  num_urls = len(urls)
  for url in urls :
    yield (url, rank / num_urls)

contribs = links.join(ranks).flatMap(
              lambda url_urls_rank : computeContribs(url_urls_rank[1][0], url_urls_rank[1][1]))
contribs.collect()

Out[9]: [('PageB', 0.5),
 ('PageC', 0.5),
 ('PageC', 1.0),
 ('PageC', 1.0),
 ('PageA', 1.0)]

In [0]:
new_rank = contribs.reduceByKey(lambda x, y : x+y).mapValues(lambda rank : 0.15 + 0.85*rank).collect()
print(new_rank)

[('PageB', 0.575), ('PageA', 1.0), ('PageC', 2.275)]